# GFK

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#adaptacion
from utils.adaptacion import gfk_grid_search, transform_gfk

#clasificadores
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[0]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/amazon/gfk/me2\_[dimensiones]\_[dominio\_fuente]_[dominio\_objetivo].pkl

# Pruebas con el dataset Amazon (3000 Dimensiones)

In [3]:
dims = 3000

In [4]:
print tipo
print dataset_name
print dims
print data_path

gfk
amazon
3000
data


In [5]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [6]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'dims': [int(dims/4), int(dims/2)],
    'n_subs': [10, 20, 50]
}

pairs = list(itertools.permutations(domains, 2))

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%s - %s" % (src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()


    gfk, score = gfk_grid_search(X_src, y_src, X_tgt, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)
    
    best_model_save_path = os.path.join(folder_path, prefix)
    print "\tGuardando modelo en %s" % best_model_save_path
    joblib.dump(gfk, best_model_save_path)

print "\nAdaptaciones completadas"

Adaptando entre:
electronics - dvd
	n: 10 - d: 750

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 0.866833220414
	n: 20 - d: 750 0.871211804538
	n: 50 - d: 750 0.884318009859
	n: 10 - d: 1500 0.866833220414
	n: 20 - d: 1500 0.871211804538
	n: 50 - d: 1500 0.884318009859
	Mejor modelo: 0.884
	Guardando modelo en models/amazon/gfk/me2_3000_electronics_dvd.pkl
electronics - kitchen
	n: 10 - d: 750 0.863756091547
	n: 20 - d: 750 0.87628008281
	n: 50 - d: 750 0.878692317083
	n: 10 - d: 1500 0.863756091547
	n: 20 - d: 1500 0.87628008281
	n: 50 - d: 1500 0.878692317083
	Mejor modelo: 0.879
	Guardando modelo en models/amazon/gfk/me2_3000_electronics_kitchen.pkl
electronics - books
	n: 10 - d: 750 0.858340137951
	n: 20 - d: 750 0.869646331552
	n: 50 - d: 750 0.870459531653
	n: 10 - d: 1500 0.858340137951
	n: 20 - d: 1500 0.869646331552
	n: 50 - d: 1500 0.870459531653
	Mejor modelo: 0.870
	Guardando modelo en models/amazon/gfk/me2_3000_electronics_books.pkl
dvd - electronics
	n: 10 - d: 750 0.771406885248
	n: 20 - d: 750 0.818094489012
	n: 50 - d: 750 0.838472516627
	n: 10 - d: 1500 0.77140

### Pruebas de clasificación

In [7]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### GFK ####
    #############
    # se adaptan los dominios usando GFK
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)

    gfk = joblib.load(best_model_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = transform_gfk(X_tr, gfk)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = transform_gfk(X_ts, gfk)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1

print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [8]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,GFK,e->d,electronics,dvd,15.762894,24.378109,8.615215
1,GFK,e->k,electronics,kitchen,8.587715,10.792770,2.205055
2,GFK,e->b,electronics,books,15.157879,28.463212,13.305333
3,GFK,d->e,dvd,electronics,11.127778,22.468062,11.340284
4,GFK,d->k,dvd,kitchen,8.587715,13.340334,4.752619
5,GFK,d->b,dvd,books,15.157879,23.858096,8.700218
6,GFK,k->e,kitchen,electronics,11.127778,15.070377,3.942599
7,GFK,k->d,kitchen,dvd,15.762894,26.263157,10.500263
8,GFK,k->b,kitchen,books,15.157879,31.805795,16.647916
9,GFK,b->e,books,electronics,11.127778,22.500563,11.372784


In [9]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/gfk/me2_3000.csv
Resultados guardados.


# Pruebas con el dataset Amazon (1000 Dimensiones)

In [10]:
dims = 1000

In [11]:
print tipo
print dataset_name
print dims
print data_path

gfk
amazon
1000
data


In [12]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [13]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'dims': [int(dims/4), int(dims/2)],
    'n_subs': [10, 20, 50]
}

pairs = list(itertools.permutations(domains, 2))

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%s - %s" % (src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()


    gfk, score = gfk_grid_search(X_src, y_src, X_tgt, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)
    
    best_model_save_path = os.path.join(folder_path, prefix)
    print "\tGuardando modelo en %s" % best_model_save_path
    joblib.dump(gfk, best_model_save_path)

print "\nAdaptaciones completadas"

Adaptando entre:
electronics - dvd
	n: 10 - d: 250 0.789418507101
	n: 20 - d: 250 0.802992277378
	n: 50 - d: 250 0.826819586153
	n: 10 - d: 500 0.789418507101
	n: 20 - d: 500 0.802992277378
	n: 50 - d: 500 0.826819586153
	Mejor modelo: 0.827
	Guardando modelo en models/amazon/gfk/me2_1000_electronics_dvd.pkl
electronics - kitchen
	n: 10 - d: 250 0.796739405409
	n: 20 - d: 250 0.798973804977
	n: 50 - d: 250 0.826867819544
	n: 10 - d: 500 0.797922344073
	n: 20 - d: 500 0.798973804977
	n: 50 - d: 500 0.826867819544
	Mejor modelo: 0.827
	Guardando modelo en models/amazon/gfk/me2_1000_electronics_kitchen.pkl
electronics - books
	n: 10 - d: 250 0.787160454876
	n: 20 - d: 250 0.793803729211
	n: 50 - d: 250 0.82521107575
	n: 10 - d: 500 0.787160454876
	n: 20 - d: 500 0.793983538023
	n: 50 - d: 500 0.82521107575
	Mejor modelo: 0.825
	Guardando modelo en models/amazon/gfk/me2_1000_electronics_books.pkl
dvd - electronics
	n: 10 - d: 250 0.718384242762
	n: 20 - d: 250 0.748283429803
	n: 50 - d: 25

### Pruebas de clasificación

In [14]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### GFK ####
    #############
    # se adaptan los dominios usando GFK
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)

    gfk = joblib.load(best_model_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = transform_gfk(X_tr, gfk)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = transform_gfk(X_ts, gfk)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1

print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [15]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,GFK,e->d,electronics,dvd,16.900423,29.410735,12.510313
1,GFK,e->k,electronics,kitchen,16.215405,17.605440,1.390035
2,GFK,e->b,electronics,books,23.710593,35.065877,11.355284
3,GFK,d->e,dvd,electronics,16.397910,23.150579,6.752669
4,GFK,d->k,dvd,kitchen,16.215405,18.827971,2.612565
5,GFK,d->b,dvd,books,23.710593,26.540664,2.830071
6,GFK,k->e,kitchen,electronics,16.397910,18.347959,1.950049
7,GFK,k->d,kitchen,dvd,16.900423,28.580715,11.680292
8,GFK,k->b,kitchen,books,23.710593,33.625841,9.915248
9,GFK,b->e,books,electronics,16.397910,25.715643,9.317733


In [16]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/gfk/me2_1000.csv
Resultados guardados.
